# 🧱 Lab: Cloud APIs and AI News Briefing Generator

**Module 1: Setup & Working Style for LLM Apps** | **Duration: ~1 hour** | **Type: Wall Lab**

---

## Learning Objectives

By the end of this lab, you will be able to:

1. **Apply** API key management and secrets hygiene practices across multiple LLM providers
2. **Build** a reusable web scraper to collect live data from news websites
3. **Create** an AI-powered News Briefing Generator using OpenAI and Anthropic
4. **Integrate** multiple LLM providers to perform different tasks on the same data

## Concepts Covered

| Concept | From |
|---------|------|
| API Keys & Environment Variables | mini-env-setup |
| Secrets Hygiene | mini-env-setup |

## Prerequisites

- **mini-env-setup**: IDE setup, uv package manager, `.env` configuration with API keys
- A payment method for API signups (this lab costs under $0.05 in API calls)

## 1. Getting Your OpenAI API Key (~5 min)

Follow these steps to get your OpenAI API key:

### Step 1: Create an Account

1. Go to [platform.openai.com](https://platform.openai.com)
2. Click **Sign up** (or **Log in** if you already have an account)
3. You can sign up with email, Google, Microsoft, or Apple

### Step 2: Add a Payment Method

1. Once logged in, click the **gear icon** (Settings) in the sidebar
2. Navigate to **Billing**
3. Click **Add payment method** and enter your card details
4. Set a monthly usage limit (recommended: **$5 to $10** for learning)

> **Tip:** OpenAI charges per API call based on tokens used. This entire lab costs less than $0.05.

### Step 3: Create an API Key

1. In the sidebar, click **API Keys**
2. Click **Create new secret key**
3. Give it a name (e.g., `ai-engineering-course`)
4. **Copy the key immediately** — you will not be able to see it again!

### Step 4: Save to Your .env File

```bash
# Add this line to your .env file
OPENAI_API_KEY=sk-your-key-here
```

> **Warning:** Never share your API key. Never commit it to git. If you accidentally expose it, rotate it immediately from the dashboard.

## 2. Getting Your Anthropic (Claude) API Key (~5 min)

### Step 1: Create an Account

1. Go to [console.anthropic.com](https://console.anthropic.com)
2. Click **Sign up**
3. Verify your email address

### Step 2: Add a Payment Method

1. Once logged in, navigate to **Settings** in the sidebar
2. Go to **Billing**
3. Add a credit or debit card
4. Anthropic may offer free credits for new accounts

### Step 3: Create an API Key

1. In the sidebar, click **API Keys**
2. Click **Create Key**
3. Give it a name (e.g., `ai-engineering-course`)
4. **Copy the key immediately**

### Step 4: Save to Your .env File

```bash
# Add this line to your .env file (below your OpenAI key)
ANTHROPIC_API_KEY=sk-ant-your-key-here
```

## 3. Verify Your .env File

Your `.env` file (in your project root) should now have both keys:

```bash
# .env — NEVER commit this to git!
OPENAI_API_KEY=sk-your-key-here
ANTHROPIC_API_KEY=sk-ant-your-key-here
```

Make sure `.env` is listed in your `.gitignore` file (covered in **mini-env-setup**).

## 4. Setup and Imports (~2 min)

In [1]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display

# Helper function to render LLM output as formatted markdown
def md(text):
    """Display text as rendered markdown."""
    display(Markdown(text))

# Load environment variables from .env
load_dotenv()

# Quick check that keys are loaded
openai_key = os.getenv("OPENAI_API_KEY")
anthropic_key = os.getenv("ANTHROPIC_API_KEY")

def mask_key(key):
    """Safely mask an API key for display."""
    if not key:
        return "NOT SET"
    if len(key) < 12:
        return "****"
    return key[:8] + "..." + key[-4:]

print("API Key Status:")
print(f"  OpenAI:    {mask_key(openai_key)}")
print(f"  Anthropic: {mask_key(anthropic_key)}")
print("\n✅ Setup complete")

API Key Status:
  OpenAI:    sk-proj-...zKoA
  Anthropic: sk-ant-a...RQAA

✅ Setup complete


## 5. Initialize and Test OpenAI Client

The OpenAI Python client automatically reads `OPENAI_API_KEY` from your environment variables — no need to pass the key manually.

In [2]:
from openai import OpenAI

# Initialize client — automatically reads OPENAI_API_KEY
client = OpenAI()
print("✅ OpenAI client initialized")

# Test the connection
def test_openai():
    """Test the OpenAI API connection."""
    print("\n🧪 Testing OpenAI API Connection")
    print("=" * 50)
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": "Say 'Hello, AI Engineer!' in exactly those words."}],
            max_tokens=20,
            temperature=0
        )
        
        print(f"✅ Connection successful!")
        print(f"Response: {response.choices[0].message.content}")
        print(f"Tokens used: {response.usage.total_tokens}")
        return True
        
    except Exception as e:
        print(f"❌ Connection failed: {type(e).__name__}")
        print(f"   Error: {str(e)[:200]}")
        return False

test_openai()

✅ OpenAI client initialized

🧪 Testing OpenAI API Connection
✅ Connection successful!
Response: Hello, AI Engineer!
Tokens used: 24


True

## 6. Initialize and Test Claude Client

The Anthropic Python client works the same way — it reads `ANTHROPIC_API_KEY` from your environment automatically.

In [3]:
import anthropic

# Initialize client — automatically reads ANTHROPIC_API_KEY
anthropic_client = anthropic.Anthropic()
print("✅ Anthropic client initialized")

# Test the connection
def test_claude():
    """Test the Anthropic Claude API connection."""
    print("\n🧪 Testing Claude API Connection")
    print("=" * 50)
    
    try:
        # Try latest model first, then fall back
        claude_models = ["claude-sonnet-4-20250514", "claude-3-5-sonnet-latest"]
        
        for model_name in claude_models:
            try:
                response = anthropic_client.messages.create(
                    model=model_name,
                    max_tokens=30,
                    messages=[{"role": "user", "content": "Say 'Hello, AI Engineer!' in exactly those words."}]
                )
                print(f"✅ Connection successful! (model: {model_name})")
                print(f"Response: {response.content[0].text}")
                print(f"Tokens used: {response.usage.input_tokens + response.usage.output_tokens}")
                return True, model_name
            except Exception:
                continue
        
        print("❌ No compatible Claude model found")
        return False, None
        
    except Exception as e:
        print(f"❌ Connection failed: {type(e).__name__}")
        print(f"   Error: {str(e)[:200]}")
        return False, None

claude_ok, claude_model = test_claude()

✅ Anthropic client initialized

🧪 Testing Claude API Connection
✅ Connection successful! (model: claude-sonnet-4-20250514)
Response: Hello, AI Engineer!
Tokens used: 28


---

## 7. Project: News Briefing Generator

Now that both API connections are working, let's build something real!

**What we are building:** An AI-powered morning briefing that:

1. **Scrapes** today's top headlines from a live news website
2. **Summarizes** the key themes using OpenAI (GPT-4o-mini)
3. **Categorizes** each story using Claude
4. **Displays** a polished morning briefing dashboard

This is a real-world pattern you will use throughout your AI engineering career:

**Data Collection -> AI Processing -> Useful Output**

### 7.1 Scraping News Headlines

We will build a **single, reusable scraping function** that works with any text-heavy news website. The function takes a URL and a CSS selector — that is it.

**Main example: [NPR Text-Only](https://text.npr.org)** — A clean, text-only news page where each story is an `<a>` tag with class `topic-title`. No JavaScript, no clutter.

**Other websites you can try** (similar simple HTML structures):

| Website | URL | CSS Selector |
|---------|-----|--------------|
| **NPR** (text-only) | `https://text.npr.org/` | `a.topic-title` |
| **CNN Lite** | `https://lite.cnn.com/` | `li > a` |
| **Lobsters** (tech news) | `https://lobste.rs/` | `a.u-url` |
| **The 68k News** | `https://68k.news/` | `li > a` |

> **Tip:** To find the right CSS selector for any site, right-click a headline in your browser, click **Inspect**, and look at the HTML tag and class wrapping each headline link.

We only need two libraries:

- **requests** — fetches the webpage HTML
- **BeautifulSoup** — parses and extracts data from the HTML

The entire scraper is roughly 15 lines of code.

In [4]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def scrape_website(url, selector="a", num_stories=10):
    """
    Scrape headlines from any news website.

    Args:
        url: The webpage URL to scrape
        selector: CSS selector to find headline links (default: "a")
        num_stories: How many stories to return (default: 10)

    Returns:
        List of dicts with rank, title, link, and source
    """
    print(f"Fetching {url}...")
    response = requests.get(url, timeout=10)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, "html.parser")

    stories = []
    for idx, item in enumerate(soup.select(selector)[:num_stories], start=1):
        title = item.text.strip()
        link = item.get("href", "")
        # Handle relative URLs
        if link and not link.startswith("http"):
            link = urljoin(url, link)
        if title:
            stories.append({
                "rank": str(idx),
                "title": title,
                "link": link,
                "source": url
            })

    return stories


# --- Scrape NPR (main example) ---
# To try a different site, swap the url and selector below.
# Examples:
#   scrape_website("https://lite.cnn.com/", selector="li > a")
#   scrape_website("https://lobste.rs/", selector="a.u-url")
#   scrape_website("https://68k.news/", selector="li > a")

stories = scrape_website(
    url="https://text.npr.org/",
    selector="a.topic-title",
    num_stories=10
)

print(f"\n{'='*50}")
print(f"Scraped {len(stories)} stories from {stories[0]['source'] if stories else 'N/A'}")
for s in stories:
    print(f"  {s['rank']:>2}. {s['title']}")

Fetching https://text.npr.org/...

Scraped 10 stories from https://text.npr.org/
   1. What you should know about Bad Bunny's Super Bowl halftime show
   2. Hate them or not, Patriots fans want the glory back in Super Bowl LX
   3. From Jesus to Jurassic Park: This year's Super Bowl ads are playing it safe
   4. He's photographed every Super Bowl. Will this one be his last?
   5. U.S. ski star Lindsey Vonn is in 'stable condition' after crash in Olympic downhill
   6. Breezy Johnson's downhill gold is America's first medal of 2026 Winter Olympics
   7. State Department will delete X posts from before Trump returned to office
   8. Immigrant whose skull was broken in 8 places during ICE arrest says beating was unprovoked
   9. 'Washington Post' CEO departs after going AWOL during massive job cuts
  10. One week since Nancy Guthrie was last seen, here's what we know


### 7.2 Generate a Morning Briefing with OpenAI

Now let's send these headlines to GPT-4o-mini and ask it to write a concise morning briefing — like a tech newsletter editor would.

In [5]:
# Prepare headlines as plain text for the LLM
headlines_text = "\n".join([f"{s['rank']}. {s['title']}" for s in stories])

print("Sending headlines to OpenAI...\n")

# Generate briefing with OpenAI
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": (
                "You are a news editor writing a concise morning briefing. "
                "Be informative and highlight what matters most."
            )
        },
        {
            "role": "user",
            "content": (
                f"Here are today's top stories:\n\n"
                f"{headlines_text}\n\n"
                f"Write a 2-3 paragraph morning briefing summarizing the key themes "
                f"and most important stories. Use markdown formatting."
            )
        }
    ],
    max_tokens=500,
    temperature=0.7
)

openai_briefing = response.choices[0].message.content

md(f"## OpenAI's Morning Briefing\n\n{openai_briefing}")

Sending headlines to OpenAI...



## OpenAI's Morning Briefing

### Morning Briefing: Key Headlines for Today

As the excitement builds for Super Bowl LX, Bad Bunny's highly anticipated halftime show is a focal point, promising a unique blend of entertainment that fans won't want to miss. Meanwhile, the New England Patriots, despite recent struggles, are rallying their fanbase with hopes of reclaiming their former glory on the big stage. This year's Super Bowl advertisements appear to tread cautiously, featuring familiar themes ranging from spiritual to nostalgic, as brands aim to resonate with a broad audience.

In sports news, former Olympic skier Lindsey Vonn is reported to be in stable condition following a crash during the Olympic downhill event, while Breezy Johnson clinched America's first gold medal at the 2026 Winter Olympics. On the political front, the State Department is set to delete Twitter posts from before Trump's return to office, stirring discussions about transparency. Additionally, the departure of the Washington Post CEO amidst significant layoffs raises questions about the future of media leadership. Lastly, the mysterious disappearance of Nancy Guthrie continues to develop, with authorities providing updates on the ongoing investigation.

### 7.3 Categorize Headlines with Claude

Now let's send the same headlines to Claude and ask it to categorize each story and pick the must-read for an AI engineer.

Notice how we use the same data but ask each model to do a **different task** — this is a common multi-model pattern.

In [6]:
print("Sending headlines to Claude...\n")

# Use the model that worked during our test, or fall back to a default
model_to_use = claude_model if claude_model else "claude-sonnet-4-20250514"

response = anthropic_client.messages.create(
    model=model_to_use,
    max_tokens=800,
    messages=[
        {
            "role": "user",
            "content": (
                f"Here are today's top stories:\n\n"
                f"{headlines_text}\n\n"
                f"For each story:\n"
                f"1. Assign a category: AI, Web Dev, Security, Business, Science, Politics, Culture, or Other\n"
                f"2. Rate its relevance for an AI engineer: High, Medium, or Low\n\n"
                f"Present this as a markdown table with columns: Title, Category, Relevance.\n\n"
                f"Then pick the number 1 must-read story for an AI engineer and explain why in 2 sentences."
            )
        }
    ]
)

claude_analysis = response.content[0].text

md(f"## Claude's Analysis\n\n{claude_analysis}")

Sending headlines to Claude...



## Claude's Analysis

| Title | Category | Relevance |
|-------|----------|-----------|
| What you should know about Bad Bunny's Super Bowl halftime show | Culture | Low |
| Hate them or not, Patriots fans want the glory back in Super Bowl LX | Culture | Low |
| From Jesus to Jurassic Park: This year's Super Bowl ads are playing it safe | Culture | Low |
| He's photographed every Super Bowl. Will this one be his last? | Culture | Low |
| U.S. ski star Lindsey Vonn is in 'stable condition' after crash in Olympic downhill | Other | Low |
| Breezy Johnson's downhill gold is America's first medal of 2026 Winter Olympics | Other | Low |
| State Department will delete X posts from before Trump returned to office | Politics | Low |
| Immigrant whose skull was broken in 8 places during ICE arrest says beating was unprovoked | Politics | Low |
| 'Washington Post' CEO departs after going AWOL during massive job cuts | Business | Medium |
| One week since Nancy Guthrie was last seen, here's what we know | Other | Low |

**Must-read story for an AI engineer:** 'Washington Post' CEO departs after going AWOL during massive job cuts

This story provides insight into major organizational changes at a leading media company that likely employs AI engineers and uses AI technology extensively in content creation and distribution. Understanding leadership instability and restructuring at tech-forward media organizations can offer valuable perspective on industry trends and potential career implications for AI professionals working in or considering the media sector.

### 7.4 Your AI Morning Briefing Dashboard

Let's combine everything into a polished, single-view dashboard.

In [18]:
from datetime import datetime

today = datetime.now().strftime("%A, %B %d, %Y")
source_name = stories[0]["source"] if stories else "News"

# Build headline links
headline_links = "\n\n".join([
    f"**{s['rank']}.** [{s['title']}]({s['link']})"
    for s in stories
])

# Display the complete briefing
md(f"""# Your Morning Briefing
Powered by Live News + OpenAI + Claude

---

## Top Stories from {source_name}

{headline_links}

---

## Summary (GPT-4o-mini)

{openai_briefing}

---

## Story Analysis (Claude Sonnet)

{claude_analysis}

""")

# Your Morning Briefing
Powered by Live News + OpenAI + Claude

---

## Top Stories from https://text.npr.org/

**1.** [What you should know about Bad Bunny's Super Bowl halftime show](https://text.npr.org/nx-s1-5698518)

**2.** [Hate them or not, Patriots fans want the glory back in Super Bowl LX](https://text.npr.org/nx-s1-5692947)

**3.** [From Jesus to Jurassic Park: This year's Super Bowl ads are playing it safe](https://text.npr.org/nx-s1-5697056)

**4.** [He's photographed every Super Bowl. Will this one be his last?](https://text.npr.org/nx-s1-5690373)

**5.** [U.S. ski star Lindsey Vonn is in 'stable condition' after crash in Olympic downhill](https://text.npr.org/nx-s1-5705859)

**6.** [Breezy Johnson's downhill gold is America's first medal of 2026 Winter Olympics](https://text.npr.org/nx-s1-5705898)

**7.** [State Department will delete X posts from before Trump returned to office](https://text.npr.org/nx-s1-5704785)

**8.** [Immigrant whose skull was broken in 8 places during ICE arrest says beating was unprovoked](https://text.npr.org/g-s1-109219)

**9.** ['Washington Post' CEO departs after going AWOL during massive job cuts](https://text.npr.org/nx-s1-5705413)

**10.** [One week since Nancy Guthrie was last seen, here's what we know](https://text.npr.org/nx-s1-5705280)

---

## Summary (GPT-4o-mini)

### Morning Briefing: Key Headlines for Today

As the excitement builds for Super Bowl LX, Bad Bunny's highly anticipated halftime show is a focal point, promising a unique blend of entertainment that fans won't want to miss. Meanwhile, the New England Patriots, despite recent struggles, are rallying their fanbase with hopes of reclaiming their former glory on the big stage. This year's Super Bowl advertisements appear to tread cautiously, featuring familiar themes ranging from spiritual to nostalgic, as brands aim to resonate with a broad audience.

In sports news, former Olympic skier Lindsey Vonn is reported to be in stable condition following a crash during the Olympic downhill event, while Breezy Johnson clinched America's first gold medal at the 2026 Winter Olympics. On the political front, the State Department is set to delete Twitter posts from before Trump's return to office, stirring discussions about transparency. Additionally, the departure of the Washington Post CEO amidst significant layoffs raises questions about the future of media leadership. Lastly, the mysterious disappearance of Nancy Guthrie continues to develop, with authorities providing updates on the ongoing investigation.

---

## Story Analysis (Claude Sonnet)

| Title | Category | Relevance |
|-------|----------|-----------|
| What you should know about Bad Bunny's Super Bowl halftime show | Culture | Low |
| Hate them or not, Patriots fans want the glory back in Super Bowl LX | Culture | Low |
| From Jesus to Jurassic Park: This year's Super Bowl ads are playing it safe | Culture | Low |
| He's photographed every Super Bowl. Will this one be his last? | Culture | Low |
| U.S. ski star Lindsey Vonn is in 'stable condition' after crash in Olympic downhill | Other | Low |
| Breezy Johnson's downhill gold is America's first medal of 2026 Winter Olympics | Other | Low |
| State Department will delete X posts from before Trump returned to office | Politics | Low |
| Immigrant whose skull was broken in 8 places during ICE arrest says beating was unprovoked | Politics | Low |
| 'Washington Post' CEO departs after going AWOL during massive job cuts | Business | Medium |
| One week since Nancy Guthrie was last seen, here's what we know | Other | Low |

**Must-read story for an AI engineer:** 'Washington Post' CEO departs after going AWOL during massive job cuts

This story provides insight into major organizational changes at a leading media company that likely employs AI engineers and uses AI technology extensively in content creation and distribution. Understanding leadership instability and restructuring at tech-forward media organizations can offer valuable perspective on industry trends and potential career implications for AI professionals working in or considering the media sector.



## 8. Challenge: Extend Your Briefing

Try these extensions on your own:

1. **Try a different source** — Swap the URL and selector to scrape CNN Lite (`https://lite.cnn.com/`, selector `li > a`) or Lobsters (`https://lobste.rs/`, selector `a.u-url`)
2. **Scrape multiple sites** — Call `scrape_website()` with different URLs and combine the results into one list
3. **Filter by topic** — Add a parameter to only show AI-related stories
4. **Sentiment analysis** — Ask the LLM to rate each headline's sentiment (positive, negative, neutral)
5. **Compare models** — Try `gpt-4o` instead of `gpt-4o-mini`. Is the briefing better? Worth the extra cost?
6. **Personalize it** — Add a system prompt that tailors the briefing to your specific interests

## 🎯 Summary

### What You Built

You built a real AI-powered application that:

- **Collects** live data from a news website using a single reusable scraper function
- **Processes** it with two different LLMs (OpenAI and Claude)
- **Produces** a useful, formatted output (Morning Briefing Dashboard)

### Key Takeaways

1. **API Key Setup** — Both OpenAI and Anthropic follow the same pattern: sign up, add payment, create key, save to `.env`
2. **Client Initialization** — Both clients auto-read API keys from environment variables
3. **Web Scraping** — `requests` + `BeautifulSoup` is all you need for simple HTML sites. One function with a URL and CSS selector can scrape many different websites.
4. **Multi-Model Usage** — Different models can serve different roles (summarization vs. classification)
5. **Real-World Pattern** — Data Collection, then AI Processing, then Useful Output is a core AI engineering workflow

### Next Steps

- **mini-ollama-setup**: Set up local LLMs with Ollama (free, private, no API key needed)
- **lab-hello-llm**: Build your first LLM-powered app with model switching